In [17]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging
from tqdm import tqdm

In [18]:
# Set the project name (this is the folder name where the output files will be stored)
custom_data_folder = 'amsterdam_custom_samples'

In [19]:
## Number of TFRecords in a single file
batch_size = 100

In [20]:
serval_data_folder = "../data"

In [21]:
input_selected_classes_filepath = serval_data_folder + '/' + custom_data_folder + '/csv_files/input_selected_classes.csv'
output_class_mapping_filepath   = serval_data_folder + '/' + custom_data_folder + '/csv_files/output_class_mapping.csv' 
output_tfrecords_train_path     = serval_data_folder + '/' + custom_data_folder + '/tfrecords_model_input/train'
output_tfrecords_eval_path      = serval_data_folder + '/' + custom_data_folder + '/tfrecords_model_input/eval'

tfrecords_train_search_string   = serval_data_folder + '/' + custom_data_folder + '/tfrecords_all/train_*.tfrecord'
tfrecords_eval_search_string    = serval_data_folder + '/' + custom_data_folder + '/tfrecords_all/eval_*.tfrecord'

In [22]:
print('[INFO]: When proportion_xxx_samples > 1.0 than samples are duplicated')
print('[INFO]: When proportion_xxx_samples < 1.0 than samples are reduced for this class')
print('')
print('[INFO]: Re-run the notebook a couple of times until you get the amounts per class you would like to see')
print('')
df_input_selected_classes = pd.read_csv(input_selected_classes_filepath, sep=";")
print('[INFO]: df_input_selected_classes')
display(df_input_selected_classes)

[INFO]: When proportion_xxx_samples > 1.0 than samples are duplicated
[INFO]: When proportion_xxx_samples < 1.0 than samples are reduced for this class

[INFO]: Re-run the notebook a couple of times until you get the amounts per class you would like to see

[INFO]: df_input_selected_classes


,label,new_label,proportion_train_samples,proportion_eval_samples,description,enable
0,1000,0,1.0,1.0,3 Distance 9mm gun shots,1
1,1001,1,1.0,1.0,556 gunshots,1
2,1002,2,4.0,1.0,Brommer Alarm,1
3,1003,3,1.0,1.0,Brommer,1
4,1004,4,1.0,1.0,Claxons,1
5,1005,5,1.0,1.0,Dichtklappende autodeuren,1
6,1006,6,1.0,1.0,"Luide mensen (schreeuwen, enz)",1
7,1007,7,1.0,1.0,Motoren,1
8,1008,8,1.0,1.0,Terrasgeluide (stille mensen),1
9,1009,9,1.0,1.0,Versterkte muziek,1


In [23]:
## Create class mapping to 1:20

## First validate that there are no duplicates in the input file
assert(not any(df_input_selected_classes.label.duplicated())) ##, "df_input_selected_classes.label is expected to have unique labels!")

## Create class mapping
df_output_class_mapping = df_input_selected_classes.loc[df_input_selected_classes.enable == 1].copy(deep=True)

df_output_class_mapping['old_label'] = df_output_class_mapping['label']
df_output_class_mapping['label'] = df_output_class_mapping['new_label']
## TODO: df_output_class_mapping['label'] = list(range(0, len(df_output_class_mapping)))
df_output_class_mapping['total_count_train'] = 0
df_output_class_mapping['expected_count_train'] = 0
df_output_class_mapping['actual_count_train'] = 0

df_output_class_mapping['total_count_eval'] = 0
df_output_class_mapping['expected_count_eval'] = 0
df_output_class_mapping['actual_count_eval'] = 0

del df_output_class_mapping['enable']
del df_output_class_mapping['new_label']

print('[INFO]: df_output_class_mapping')
display(df_output_class_mapping)

[INFO]: df_output_class_mapping


,label,proportion_train_samples,proportion_eval_samples,description,old_label,total_count_train,expected_count_train,actual_count_train,total_count_eval,expected_count_eval,actual_count_eval
0,0,1.0,1.0,3 Distance 9mm gun shots,1000,0,0,0,0,0,0
1,1,1.0,1.0,556 gunshots,1001,0,0,0,0,0,0
2,2,4.0,1.0,Brommer Alarm,1002,0,0,0,0,0,0
3,3,1.0,1.0,Brommer,1003,0,0,0,0,0,0
4,4,1.0,1.0,Claxons,1004,0,0,0,0,0,0
5,5,1.0,1.0,Dichtklappende autodeuren,1005,0,0,0,0,0,0
6,6,1.0,1.0,"Luide mensen (schreeuwen, enz)",1006,0,0,0,0,0,0
7,7,1.0,1.0,Motoren,1007,0,0,0,0,0,0
8,8,1.0,1.0,Terrasgeluide (stille mensen),1008,0,0,0,0,0,0
9,9,1.0,1.0,Versterkte muziek,1009,0,0,0,0,0,0


In [24]:
def createTFRecord(example_old, new_labels, old_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})
    #print("New labels: " + str(new_labels))
    
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    global_label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=old_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat, "global_labels": global_label_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [25]:
def getNewLabels(df_label_mapping, old_labels):
    return(df_label_mapping.loc[df_label_mapping.old_label.isin(old_labels)].label.values)

In [26]:
def getAllTFRecords(tfrecords_search_string, df_class_mapping, count_type):
    tfrecords = {}
    
    ## Get files
    files = gfile.Glob(tfrecords_search_string)
    print('[INFO]: Processing ', len(list(files)), ' files from directory: ', tfrecords_search_string, sep='')
    
    for file in tqdm(files):
        records = list(tf.python_io.tf_record_iterator(path = file))
        print('[INFO]: Processing: ', len(records), ' tfrecords from file: ', file, sep='')

        for string_record in records:
            old_tfrecord = tf.train.SequenceExample()
            old_tfrecord.ParseFromString(string_record)

            ## Get labels and find for each label the m-code in class_label_indices_old
            old_labels = old_tfrecord.context.feature['labels'].int64_list.value
            new_labels = getNewLabels(df_class_mapping, old_labels)
            if len(new_labels) == 0:
                ## Skip files without valid labels
                continue
                
            new_tfrecord = createTFRecord(old_tfrecord, new_labels, old_labels)

            ## Update count of found labels
            df_class_mapping[count_type] += np.where(df_class_mapping.old_label.isin(old_labels), 1, 0)

            ## Store new record in dict
            tfrecords[len(tfrecords)] = new_tfrecord
            
    return({'tfrecords':tfrecords, 'df_class_mapping':df_class_mapping})

In [27]:
def writeTFRecordsInBatches(df_class_mapping, tfrecords, batch_size, output_directory, expect_count_param, actual_count_param):
    ## Generate random permutation without replace
    ## Generate multiple to oversample samples if needed
    permutations = [random.sample(range(0, len(tfrecords)), len(tfrecords)), 
                    random.sample(range(0, len(tfrecords)), len(tfrecords)),
                    random.sample(range(0, len(tfrecords)), len(tfrecords)),
                    random.sample(range(0, len(tfrecords)), len(tfrecords))]
    
    batch_index = 0
    tfrecords_remain_in_batch = -1
    for permutation in permutations:
        for index in tqdm(permutation):
            if tfrecords_remain_in_batch == 0:
                batch_writer.close()

            if tfrecords_remain_in_batch <= 0:
                ## Create new batch file
                tfrecords_remain_in_batch = batch_size
                batch_index = batch_index + 1
                batch_file_name = str(output_directory + '/sample_' + str(batch_index) + '.tfrecord')
                batch_writer = tf.python_io.TFRecordWriter(batch_file_name)

            # Update total number of selected samples and check if we should add this one
            tfrecord = tfrecords[index]
            global_labels = tfrecord.context.feature['global_labels'].int64_list.value

            # Skip samples with classes that we already have enough
            if not np.any(df_class_mapping.loc[df_class_mapping[expect_count_param] > df_class_mapping[actual_count_param]]
                                          .old_label.isin(global_labels)):
                continue

            ## Update count of found labels
            df_class_mapping[actual_count_param] += np.where(df_class_mapping.old_label.isin(global_labels), 1, 0)

            ## Write to file
            batch_writer.write(tfrecord.SerializeToString())
            tfrecords_remain_in_batch = tfrecords_remain_in_batch - 1

    batch_writer.close()
    return(df_class_mapping)

In [29]:
def processTFRecords(df_output_class_mapping, output_tfrecords_path, tfrecords_search_string, suffix):
    total_count_param      = 'total_count_' + suffix
    expected_count_param   = 'expected_count_' + suffix
    actual_count_param     = 'actual_count_' + suffix
    proportion_count_param = 'proportion_' + suffix + '_samples'

    ## Read all Records
    tfrecords = getAllTFRecords(tfrecords_search_string, df_output_class_mapping.copy(deep=True), total_count_param)

    # Set back to generic data frame
    df_output_class_mapping = tfrecords['df_class_mapping']
    df_output_class_mapping[expected_count_param] = df_output_class_mapping[proportion_count_param] * df_output_class_mapping[total_count_param]

    ## Write batches
    df_output_class_mapping = writeTFRecordsInBatches(df_output_class_mapping, tfrecords['tfrecords'], batch_size, output_tfrecords_path, expected_count_param, actual_count_param)
    return(df_output_class_mapping)

In [30]:
df_output_class_mapping = processTFRecords(df_output_class_mapping, output_tfrecords_train_path, tfrecords_train_search_string, 'train')

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]: Processing 1 files from directory: ../data/amsterdam_custom_samples/tfrecords_all/train_*.tfrecord
[INFO]: Processing: 13888 tfrecords from file: ..\data\amsterdam_custom_samples\tfrecords_all\train_20200328.tfrecord


100%|██████████| 13888/13888 [00:08<00:00, 1732.58it/s]


In [32]:
df_output_class_mapping = processTFRecords(df_output_class_mapping, output_tfrecords_eval_path, tfrecords_eval_search_string, 'eval')

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]: Processing 1 files from directory: ../data/amsterdam_custom_samples/tfrecords_all/eval_*.tfrecord
[INFO]: Processing: 13528 tfrecords from file: ..\data\amsterdam_custom_samples\tfrecords_all\eval_20200328.tfrecord


100%|██████████| 13528/13528 [00:08<00:00, 1639.10it/s]


In [33]:
display(df_output_class_mapping)

,label,proportion_train_samples,proportion_eval_samples,description,old_label,total_count_train,expected_count_train,actual_count_train,total_count_eval,expected_count_eval,actual_count_eval
0,0,1.0,1.0,3 Distance 9mm gun shots,1000,0,0.0,0,0,0.0,0
1,1,1.0,1.0,556 gunshots,1001,1712,1712.0,1712,1792,1792.0,1792
2,2,4.0,1.0,Brommer Alarm,1002,44,176.0,176,48,48.0,48
3,3,1.0,1.0,Brommer,1003,1676,1676.0,1676,1572,1572.0,1572
4,4,1.0,1.0,Claxons,1004,260,260.0,260,268,268.0,268
5,5,1.0,1.0,Dichtklappende autodeuren,1005,340,340.0,340,316,316.0,316
6,6,1.0,1.0,"Luide mensen (schreeuwen, enz)",1006,1320,1320.0,1320,1216,1216.0,1216
7,7,1.0,1.0,Motoren,1007,108,108.0,108,96,96.0,96
8,8,1.0,1.0,Terrasgeluide (stille mensen),1008,1124,1124.0,1124,1020,1020.0,1020
9,9,1.0,1.0,Versterkte muziek,1009,76,76.0,76,60,60.0,60


In [ ]:
## Write new labels and descriptions back to csv_file
df_output_class_mapping['display_name'] = df_output_class_mapping.description
df_output_class_mapping['index'] = df_output_class_mapping.label
df_output_class_mapping.to_csv(output_class_mapping_filepath, sep = ";")